In [1]:
!pip install python-mnist

In [2]:
%tensorflow_version 2.x

# use this to work with tensorflow 2.x on colab

In [3]:
!wget https://www.dropbox.com/s/y54njjo7w117bon/mnist_data.zip

--2021-01-29 13:22:28--  https://www.dropbox.com/s/y54njjo7w117bon/mnist_data.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.18, 2620:100:6023:18::a27d:4312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/y54njjo7w117bon/mnist_data.zip [following]
--2021-01-29 13:22:28--  https://www.dropbox.com/s/raw/y54njjo7w117bon/mnist_data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd50fb0c81d9200fddc53b19e03.dl.dropboxusercontent.com/cd/0/inline/BH6Y1PHmHti27XHppyLaHAVnT46x3x44SVdXqKXOFT3IbiTd5RYjStWBk-L4U3b_PEhwTEdTUee3Ix5qFvFKYMLKKF4MOHbfkklAN2wtl5vKKEHd6rpRN69v6Qgx826kxKg/file# [following]
--2021-01-29 13:22:28--  https://ucd50fb0c81d9200fddc53b19e03.dl.dropboxusercontent.com/cd/0/inline/BH6Y1PHmHti27XHppyLaHAVnT46x3x44SVdXqKXOFT3IbiTd5RYjStWBk-L4U3b_PEhwTEdTUee3Ix5qFvFKYMLKKF4MOHbfkklAN2

In [4]:
!unzip mnist_data

Archive:  mnist_data.zip
   creating: mnist_data/
  inflating: mnist_data/t10k-images-idx3-ubyte  
  inflating: mnist_data/t10k-labels-idx1-ubyte  
  inflating: mnist_data/train-images-idx3-ubyte  
  inflating: mnist_data/train-labels-idx1-ubyte  


In [45]:
import warnings
warnings.filterwarnings('ignore')

In [46]:
from mnist import MNIST
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

mndata = MNIST(r'/content/mnist_data')

In [47]:
images_train, labels_train = mndata.load_training()
images_test,labels_test=mndata.load_testing()

In [48]:
labels_train=np.array(labels_train).reshape(-1,1)
labels_test=np.array(labels_test).reshape(-1,1)

In [49]:
onehot=OneHotEncoder()

onehot.fit(labels_train)

labels_train=onehot.transform(labels_train)
labels_test=onehot.transform(labels_test)
labels_train=labels_train.toarray()
labels_test=labels_test.toarray()

In [50]:
images_train=np.array(images_train).reshape(60000,784)/255
images_test=np.array(images_test).reshape(10000,784)/255

In [51]:
images_train=images_train.reshape(60000,  28, 28,1).astype('float32')
images_test=images_test.reshape(10000, 28, 28,1).astype('float32')

In [ ]:
# Functional API

In [52]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D

In [53]:


image=Input(shape=(28,28,1))

conv1=Conv2D(32, (5, 5), activation='relu',padding='same')(image)
pool1=MaxPooling2D(pool_size=(2, 2))(conv1)

conv2=Conv2D(64, (5, 5),activation='relu',padding='same')(pool1)
pool2=MaxPooling2D(pool_size=(2, 2))(conv2)

flat_layer=Flatten()(pool2)

dense1=Dense(1024, activation='relu')(flat_layer)
drop1=Dropout(0.2)(dense1)

dense2=Dense(200, activation='relu')(drop1)
drop2=Dropout(0.2)(dense2)

output=Dense(10, activation='softmax')(drop2)

model=Model(inputs=image,outputs=output)

In [54]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 1024)              3212

In [55]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [58]:
model.fit(images_train,labels_train,validation_data=(images_test,labels_test),
         epochs=10,batch_size=50,verbose=1)

Epoch 1/10
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0157 - accuracy: 0.9955 - val_loss: 0.0391 - val_accuracy: 0.9918
Epoch 2/10
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0103 - accuracy: 0.9971 - val_loss: 0.0412 - val_accuracy: 0.9911
Epoch 3/10
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0111 - accuracy: 0.9970 - val_loss: 0.0498 - val_accuracy: 0.9911
Epoch 4/10
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0076 - accuracy: 0.9977 - val_loss: 0.0372 - val_accuracy: 0.9923
Epoch 5/10
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0081 - accuracy: 0.9976 - val_loss: 0.0422 - val_accuracy: 0.9908
Epoch 6/10
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0097 - accuracy: 0.9975 - val_loss: 0.0410 - val_accuracy: 0.9923
Epoch 7/10
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0071 - accuracy: 0.9983 - val_loss: 0.0311 -

In [ ]:
# Saving the model 

In [59]:
from tensorflow.keras.models import model_from_json

In [60]:
model_json = model.to_json()

In [61]:
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [62]:
model.save_weights("model.h5")